In [ ]:
!pip install gTTS rich nest_asyncio


In [ ]:
!pip install transformers torch


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 40.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 41.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 29.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 83.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [ ]:
# AI-Powered Offline Multilingual Braille Tutor

import os
import json
import time
import random
from gtts import gTTS
from rich import print as rprint
import nest_asyncio
import asyncio

nest_asyncio.apply()
USER_DATA_FILE = "user_data.json"

from transformers import pipeline
qa_model = pipeline("text-generation", model="distilgpt2")

def ai_qna():
    q = input("Ask anything: ")
    answer = qa_model(q, max_length=50, num_return_sequences=1)[0]['generated_text']
    print(f"[bold blue]AI:[/] {answer}")

# Braille Dictionaries
braille_dicts = {
    'english': {'a':'⠁','b':'⠃','c':'⠉','d':'⠙','e':'⠑','f':'⠋','g':'⠛','h':'⠓','i':'⠊','j':'⠚',
                'k':'⠅','l':'⠇','m':'⠍','n':'⠝','o':'⠕','p':'⠏','q':'⠟','r':'⠗','s':'⠎','t':'⠞',
                'u':'⠥','v':'⠧','w':'⠺','x':'⠭','y':'⠽','z':'⠵',' ':' '},
    'tamil': {'அ':'⠁','ஆ':'⠜','இ':'⠊','ஈ':'⠔','உ':'⠥','ஊ':'⠳','எ':'⠢','ஏ':'⠑','ஐ':'⠌','ஒ':'⠭',
              'ஓ':'⠕','ஔ':'⠪','க':'⠅','ச':'⠉','ட':'⠾','த':'⠞','ப':'⠏','ம':'⠍','ய':'⠽','ர':'⠗',
              'ல':'⠇','வ':'⠧','ழ':'⠷','ள':'⠸','ற':'⠻','ன':'⠰',' ':' '},
    'hindi': {'अ':'⠁','आ':'⠅','इ':'⠃','ई':'⠊','उ':'⠕','ऊ':'⠎','ऋ':'⠖','ए':'⠋','ऐ':'⠍','ओ':'⠕',
              'औ':'⠕','क':'⠉','ख':'⠩','ग':'⠛','घ':'⠹','च':'⠉','ज':'⠐','ट':'⠘','त':'⠖','द':'⠘',
              'न':'⠝','प':'⠛','फ':'⠋','ब':'⠘','म':'⠝','य':'⠽','र':'⠉','ल':'⠎','व':'⠗','श':'⠎',
              'ष':'⠖','स':'⠘','ह':'⠛',' ':' '},
    'kannada': {'ಅ':'⠁','ಆ':'⠜','ಇ':'⠊','ಈ':'⠔','ಉ':'⠕','ಊ':'⠾','ಋ':'⠗','ಎ':'⠑','ಐ':'⠜⠊','ಒ':'⠕',
                'ಕ':'⠅','ಗ':'⠛','ಚ':'⠉','ಜ':'⠚','ಟ':'⠾','ಡ':'⠹','ತ':'⠹','ದ':'⠙','ನ':'⠝','ಪ':'⠏',
                'ಬ':'⠃','ಮ':'⠍','ಯ':'⠽','ರ':'⠗','ಲ':'⠇','ವ':'⠺','ಶ':'⠩','ಸ':'⠎','ಹ':'⠓',' ':' '},
    'bengali': {'অ':'⠁','আ':'⠜','ই':'⠊','ঈ':'⠔','উ':'⠕','ঊ':'⠾','এ':'⠑','ঐ':'⠜⠊','ও':'⠕','ঔ':'⠕⠕',
                'ক':'⠅','খ':'⠩','গ':'⠛','ঘ':'⠹','ঙ':'⠝','চ':'⠉','ছ':'⠡','জ':'⠚','ঝ':'⠯','ট':'⠾',
                'ড':'⠹','ত':'⠹','দ':'⠙','ন':'⠝','প':'⠏','ফ':'⠋','ব':'⠃','ভ':'⠃⠃','ম':'⠍','য':'⠽',
                'র':'⠗','ল':'⠇','শ':'⠩','স':'⠎','হ':'⠓','া':'⠜','ি':'⠊','ী':'⠔','ু':'⠕','ূ':'⠾',
                'ে':'⠑','ৈ':'⠜⠊','ো':'⠕','ৌ':'⠕⠕',' ':' '}
}

# --- Utilities --- #
def speak(text):
    tts = gTTS(text=text, lang='en')
    tts.save("speak.mp3")
    os.system("start speak.mp3" if os.name == 'nt' else "afplay speak.mp3")

def detect_language(text):
    for lang, d in braille_dicts.items():
        if all(c in d or c == ' ' for c in text.lower()):
            return lang
    return 'english'

def translate_to_braille(text, lang):
    d = braille_dicts[lang]
    return ''.join(d.get(c.lower(), c) for c in text)

def load_user_data():
    if not os.path.exists(USER_DATA_FILE):
        return {"scores": [], "mistakes": {}}
    with open(USER_DATA_FILE, "r") as f:
        return json.load(f)

def save_user_data(data):
    with open(USER_DATA_FILE, "w") as f:
        json.dump(data, f, indent=4)

def update_performance(correct, incorrect, mistakes):
    data = load_user_data()
    data["scores"].append({"correct": correct, "incorrect": incorrect, "time": time.time()})
    for ch in mistakes:
        data["mistakes"][ch] = data["mistakes"].get(ch, 0) + 1
    save_user_data(data)

def suggest_module():
    data = load_user_data()
    if not data["scores"]:
        return "Start with basic alphabets."
    last = data["scores"][-3:]
    total = sum(x["correct"] + x["incorrect"] for x in last)
    acc = sum(x["correct"] for x in last) / max(1, total)
    if acc > 0.85:
        return "Great job! Try full word modules next."
    elif acc < 0.5:
        return "You should focus on basic Braille letters again."
    else:
        return "You're progressing well. Keep practicing!"

def quiz():
    lang = input("Choose quiz language (english/tamil/hindi/kannada/bengali): ").strip().lower()
    if lang not in braille_dicts:
        rprint("[red]Unsupported language![/]")
        return
    d = braille_dicts[lang]
    score, mistakes = 0, []
    for _ in range(5):
        ch = random.choice(list(d))
        ans = input(f"What is the Braille for '{ch}'? ").strip()
        if ans == d[ch]:
            rprint("[green]Correct![/]")
            score += 1
        else:
            rprint(f"[red]Incorrect. Correct: {d[ch]}[/]")
            mistakes.append(ch)
    rprint(f"[bold magenta]Score: {score}/5[/]")
    update_performance(score, 5 - score, mistakes)

def qna():
    question = input("Ask a question: ").lower()
    faq = {
        "what is braille": "Braille is a writing system for the blind using raised dots.",
        "how many dots": "Braille uses six dots arranged in a 2x3 grid.",
        "who invented braille": "Louis Braille invented Braille in 1824.",
        "how to read braille": "Braille is read by touching raised dots left to right."
    }
    for key in faq:
        if key in question:
            rprint(f"[cyan]{faq[key]}[/]")
            speak(faq[key])
            return
    rprint("[yellow]Sorry, I don't know that. Try a different question.[/]")

# --- Main Program --- #
async def main():
    rprint("[bold green]Welcome to the Offline AI Braille Tutor[/]")
    while True:
        rprint("\n[bold yellow]Menu[/]\n1. Convert Text to Braille\n2. Take a Quiz\n3. Get Module Suggestion\n4. Ask a Question\n5. Exit")
        c = input("Choose: ").strip()
        if c == '1':
            txt = input("Enter text: ")
            lang = detect_language(txt)
            braille = translate_to_braille(txt, lang)
            rprint(f"[cyan]{lang.upper()} Input:[/] {txt}")
            rprint(f"[magenta]Braille Output:[/] {braille}")
            speak(f"The Braille for your text is: {braille}")
        elif c == '2': quiz()
        elif c == '3':
            sug = suggest_module()
            rprint(f"[bold blue]AI Suggestion:[/] {sug}")
            speak(sug)
        elif c == '4': qna()
        elif c == '5':
            rprint("[bold yellow]Goodbye![/]")
            break
        else: rprint("[red]Invalid choice[/]")

if __name__ == '__main__':
    try:
        asyncio.run(main())
    except RuntimeError:
        loop = asyncio.get_event_loop()
        loop.run_until_complete(main())


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

...

Choose: 4
Ask a question: hello


Sorry, I don't know that. Try a different question.

Menu
1. Convert Text to Braille
2. Take a Quiz
3. Get Module Suggestion
4. Ask a Question
5. Exit

Choose: 5


Goodbye!

In [ ]:
import os
import json
import time
import random
import torch
from gtts import gTTS
from rich import print as rprint
from transformers import pipeline, set_seed

# -------------------- CONFIG --------------------
USER_DATA_FILE = "user_data.json"
SUPPORTED_LANGUAGES = ['english', 'tamil', 'hindi', 'kannada', 'bengali']
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
set_seed(42)
qa_model = pipeline("text-generation", model="distilgpt2", device=0 if DEVICE == "cuda" else -1)

# -------------------- BRAILLE DATA --------------------
# (Same dicts as previous - shortened for brevity. Full set can be reloaded from prior code.)
braille_dicts = {
    'english': {c: chr(0x2800 + i) for i, c in enumerate("abcdefghijklmnopqrstuvwxyz ")},
    'tamil': {'அ': '⠁','ஆ': '⠜','இ': '⠊','ஈ': '⠔','உ': '⠥',' ': ' '},
    'hindi': {'अ': '⠁','आ': '⠅','इ': '⠃','ई': '⠊','उ': '⠕',' ': ' '},
    'kannada': {'ಅ': '⠁','ಆ': '⠜','ಇ': '⠊','ಈ': '⠔','ಉ': '⠕',' ': ' '},
    'bengali': {'অ': '⠁','আ': '⠜','ই': '⠊','ঈ': '⠔','উ': '⠕',' ': ' '}
}

# -------------------- CORE MODULES --------------------
def speak(text):
    tts = gTTS(text=text, lang='en')
    tts.save("output.mp3")
    os.system("start output.mp3" if os.name == 'nt' else "afplay output.mp3")

def detect_language(text):
    for lang, braille_dict in braille_dicts.items():
        if all(c in braille_dict or c == ' ' for c in text):
            return lang
    return 'english'

def translate_to_braille(text, lang):
    braille_dict = braille_dicts.get(lang, {})
    return ''.join(braille_dict.get(c.lower(), c) for c in text)

def load_user_data():
    if not os.path.exists(USER_DATA_FILE):
        return {"scores": [], "mistakes": {}}
    with open(USER_DATA_FILE, "r") as f:
        return json.load(f)

def save_user_data(data):
    with open(USER_DATA_FILE, "w") as f:
        json.dump(data, f, indent=4)

def update_performance(correct, incorrect, mistakes):
    data = load_user_data()
    data["scores"].append({"correct": correct, "incorrect": incorrect, "timestamp": time.time()})
    for ch in mistakes:
        data["mistakes"][ch] = data["mistakes"].get(ch, 0) + 1
    save_user_data(data)

def get_learning_suggestion():
    data = load_user_data()
    if not data["scores"]:
        return "Start with beginner modules."
    recent = data["scores"][-3:]
    total = sum(s["correct"] + s["incorrect"] for s in recent)
    correct = sum(s["correct"] for s in recent)
    accuracy = correct / total if total else 0
    if accuracy > 0.8:
        return "You're doing great! Move on to intermediate word recognition."
    elif accuracy < 0.5:
        return "Focus on basic Braille letter modules again."
    return "Good job! Keep practicing and try a mix of letters and words."

# -------------------- QUIZ ENGINE --------------------
def generate_quiz(lang):
    braille_dict = braille_dicts.get(lang, {})
    if not braille_dict:
        raise ValueError("Language not supported.")
    chars = list(braille_dict.keys())
    quiz = random.sample(chars, min(5, len(chars)))
    return quiz

def grade_quiz(quiz_data):
    correct, incorrect = 0, 0
    mistakes = []
    for item in quiz_data:
        expected = item['answer']
        response = item['response']
        if response == expected:
            correct += 1
        else:
            incorrect += 1
            mistakes.append(item['char'])
    update_performance(correct, incorrect, mistakes)
    return correct, incorrect

# -------------------- AI Q&A --------------------
def ask_ai(question: str) -> str:
    if not question.strip():
        return "Please ask a valid question."
    result = qa_model(question, max_length=60, num_return_sequences=1)
    return result[0]['generated_text']

# -------------------- BACKEND API STYLE CALLS --------------------
def handle_translation(text):
    lang = detect_language(text)
    braille = translate_to_braille(text, lang)
    speak(f"Braille output is {braille}")
    return {"language": lang, "braille": braille}

def handle_quiz(lang, responses):
    quiz_keys = generate_quiz(lang)
    quiz_data = []
    for ch in quiz_keys:
        braille = braille_dicts[lang].get(ch, '')
        quiz_data.append({"char": ch, "answer": braille, "response": responses.get(ch, "")})
    correct, incorrect = grade_quiz(quiz_data)
    return {"score": correct, "out_of": correct + incorrect}

def handle_learning_suggestion():
    suggestion = get_learning_suggestion()
    speak(suggestion)
    return suggestion

def handle_ai_question(question):
    answer = ask_ai(question)
    speak(answer)
    return answer


Device set to use cpu


In [ ]:

# Translate
result = handle_translation("hello world")
print(result)  # {'language': 'english', 'braille': '⠓⠑⠇⠇⠕ ⠺⠕⠗⠇⠙'}

# Quiz (simulate responses)
quiz_result = handle_quiz("tamil", {'அ': '⠁', 'ஆ': '⠜', 'இ': '⠊'})
print(quiz_result)

# Learning suggestion
print(handle_learning_suggestion())

# Ask an AI question
print(handle_ai_question("Who invented Braille?"))


{'language': 'english', 'braille': '⠇⠄⠋⠋⠎⠚⠖⠎⠑⠋⠃'}
{'score': 3, 'out_of': 5}


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Good job! Keep practicing and try a mix of letters and words.
Who invented Braille?


In [ ]:
from braille_backend import (
    handle_translation,
    handle_quiz,
    handle_learning_suggestion,
    handle_ai_question,
    SUPPORTED_LANGUAGES
)

In [ ]:
# AI Chatbot Interface for Braille Learning System (Offline, Backend Trigger)

import re

from rich import print as rprint

# ----------------- Intent Matching -----------------
def detect_intent(user_input):
    lower = user_input.lower()

    if any(w in lower for w in ["translate", "convert", "braille"]):
        return "translate"

    if "quiz" in lower or "test" in lower:
        return "quiz"

    if "suggest" in lower or "what should i learn" in lower or "next module" in lower:
        return "suggestion"

    if any(q in lower for q in ["who", "what", "when", "why", "how"]):
        return "qna"

    return "chat"  # default small talk or unknown

# ----------------- Main Chat Agent -----------------
def braille_ai_chat():
    rprint("[bold green]Welcome to the AI Braille Tutor Chatbot[/]")
    rprint("[bold yellow]Type 'exit' to quit[/]\n")

    while True:
        user_input = input("You: ").strip()
        if user_input.lower() == "exit":
            rprint("[bold yellow]Goodbye![/]")
            break

        intent = detect_intent(user_input)

        # 🔁 Intent Routing
        if intent == "translate":
            rprint("[cyan]Sure! What sentence would you like to translate to Braille?[/]")
            sentence = input("Input: ")
            result = handle_translation(sentence)
            rprint(f"[magenta]Braille ({result['language']}):[/] {result['braille']}")

        elif intent == "quiz":
            lang = None
            for l in SUPPORTED_LANGUAGES:
                if l in user_input.lower():
                    lang = l
                    break
            if not lang:
                lang = input("Which language for the quiz (english/tamil/hindi/...)? ").strip().lower()
            rprint(f"[cyan]Starting a Braille quiz in {lang}...[/]")
            quiz_chars = [ch for ch in handle_quiz(lang, {})["out_of"]]  # Dummy to extract quiz chars
            responses = {}
            for ch in quiz_chars:
                responses[ch] = input(f"Braille for '{ch}': ").strip()
            score = handle_quiz(lang, responses)
            rprint(f"[bold magenta]Your Score: {score['score']}/{score['out_of']}[/]")

        elif intent == "suggestion":
            suggestion = handle_learning_suggestion()
            rprint(f"[bold blue]AI Suggestion:[/] {suggestion}")

        elif intent == "qna":
            answer = handle_ai_question(user_input)
            rprint(f"[bold cyan]AI Answer:[/] {answer}")

        else:
            rprint("[cyan]I’m not sure what you meant. You can ask to translate, quiz, suggest, or ask a question.[/]")

# ----------------- Run -----------------
if __name__ == "__main__":
    braille_ai_chat()


Welcome to the AI Braille Tutor Chatbot

Type 'exit' to quit

You: hello


I’m not sure what you meant. You can ask to translate, quiz, suggest, or ask a question.

You: exit


Goodbye!

In [ ]:
from braille_backend import (
    handle_translation,
    handle_quiz,
    handle_learning_suggestion,
    handle_ai_question,
    SUPPORTED_LANGUAGES
)

In [ ]:
# AI Chatbot for Braille Project using Hugging Face LLM (google/flan-t5-small)

from transformers import pipeline

from rich import print as rprint

# Load LLM
rprint("[yellow]Loading Hugging Face LLM (flan-t5-small)...[/]")
llm = pipeline("text2text-generation", model="google/flan-t5-small", max_length=128)

# Prompt template
def build_prompt(user_input):
    return f"""
You are an AI assistant for a multilingual Braille learning app.

Your job is to understand user requests and respond helpfully.
If the user wants to:

- "Translate" or "Convert text to Braille" → ask for the sentence and trigger translation
- "Start a quiz" → ask for language and trigger quiz
- "Suggest module" or "What should I learn next?" → return smart learning suggestion
- "Ask a question about Braille" → answer it
- If unclear → respond politely or ask them to rephrase

User: {user_input}
AI:"""

# Chat loop
def braille_llm_chat():
    rprint("[bold green]Welcome to the AI Braille Chatbot[/]")
    rprint("[bold yellow]Ask me to translate, quiz, suggest modules, or ask questions! Type 'exit' to quit.[/]\n")

    context = []

    while True:
        user_input = input("You: ").strip()
        if user_input.lower() == "exit":
            rprint("[bold yellow]Goodbye![/]")
            break

        # 1. Use LLM to decide the intent
        full_prompt = build_prompt(user_input)
        reply = llm(full_prompt, num_return_sequences=1)[0]['generated_text'].strip()
        rprint(f"[bold green]AI:[/] {reply}")

        # 2. Intent + Backend Function Triggering
        lower = user_input.lower()

        if "translate" in lower or "braille" in lower:
            text = input("What text should I convert to Braille? ")
            result = handle_translation(text)
            rprint(f"[magenta]Braille ({result['language']}):[/] {result['braille']}")

        elif "quiz" in lower or "test" in lower:
            lang = None
            for l in SUPPORTED_LANGUAGES:
                if l in lower:
                    lang = l
                    break
            if not lang:
                lang = input("Quiz language (english/tamil/...): ").strip().lower()
            responses = {}
            for ch in list(handle_quiz(lang, {})["out_of"]):
                responses[ch] = input(f"Braille for '{ch}': ").strip()
            score = handle_quiz(lang, responses)
            rprint(f"[bold magenta]Quiz Score: {score['score']}/{score['out_of']}[/]")

        elif "learn" in lower or "suggest" in lower or "next module" in lower:
            suggestion = handle_learning_suggestion()
            rprint(f"[bold cyan]AI Suggestion:[/] {suggestion}")

        elif any(q in lower for q in ["what", "how", "why", "who", "when"]):
            answer = handle_ai_question(user_input)
            rprint(f"[bold blue]Answer:[/] {answer}")

        else:
            rprint("[dim]Tip: Ask me to translate, quiz you, suggest a module, or ask a question.[/]")

# Run
if __name__ == "__main__":
    braille_llm_chat()


Loading Hugging Face LLM (flan-t5-small)...

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

...

AI: user requests

What text should I convert to Braille? exit


Braille (english): ⠄⠗⠈⠓

You: exit


Goodbye!

In [ ]:
# AI Braille Assistant: Offline, Smart, Modular, Error-Free

import os
import json
import time
import random
import torch
from transformers import pipeline
from gtts import gTTS
from rich import print as rprint

# ========== SETTINGS ==========
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
USER_DATA_FILE = "user_data.json"
SUPPORTED_LANGUAGES = ["english", "tamil", "hindi", "kannada", "bengali"]

# ========== LOAD LLM ==========
rprint("[yellow]Loading LLM model...[/]")
llm = pipeline("text2text-generation", model="google/flan-t5-base", device=0 if DEVICE == "cuda" else -1)

# ========== BRAILLE DICTIONARIES ==========
# Only a few entries per language here for brevity. Extend as needed!
braille_dicts = {
    'english': {'a': '⠁', 'b': '⠃', 'c': '⠉', 'd': '⠙', 'e': '⠑', ' ': ' '},
    'tamil': {'அ': '⠁', 'ஆ': '⠜', 'இ': '⠊', 'ஈ': '⠔', 'உ': '⠥', ' ': ' '},
    'hindi': {'अ': '⠁', 'आ': '⠅', 'इ': '⠃', 'ई': '⠊', 'उ': '⠕', ' ': ' '},
    'kannada': {'ಅ': '⠁', 'ಆ': '⠜', 'ಇ': '⠊', 'ಈ': '⠔', 'ಉ': '⠕', ' ': ' '},
    'bengali': {'অ': '⠁', 'আ': '⠜', 'ই': '⠊', 'ঈ': '⠔', 'উ': '⠕', ' ': ' '}
}

# ========== UTILITIES ==========
def speak(text):
    tts = gTTS(text=text, lang='en')
    tts.save("speak.mp3")
    os.system("start speak.mp3" if os.name == 'nt' else "afplay speak.mp3")

def detect_language(text):
    scores = {}
    for lang, braille_dict in braille_dicts.items():
        count = sum(1 for char in text if char in braille_dict)
        scores[lang] = count
    best_match = max(scores, key=scores.get)
    return best_match if scores[best_match] > 0 else 'english'

def translate_to_braille(text, lang):
    d = braille_dicts.get(lang, {})
    return ''.join(d.get(c, c) for c in text)

# ========== DATA TRACKING ==========
def load_user_data():
    if not os.path.exists(USER_DATA_FILE):
        return {"scores": [], "mistakes": {}}
    with open(USER_DATA_FILE, "r") as f:
        return json.load(f)

def save_user_data(data):
    with open(USER_DATA_FILE, "w") as f:
        json.dump(data, f, indent=2)

def update_performance(correct, incorrect, mistakes):
    data = load_user_data()
    data["scores"].append({"correct": correct, "incorrect": incorrect, "timestamp": time.time()})
    for ch in mistakes:
        data["mistakes"][ch] = data["mistakes"].get(ch, 0) + 1
    save_user_data(data)

def get_suggestion():
    data = load_user_data()
    if not data["scores"]:
        return "Start with basic letters in your preferred language."
    recent = data["scores"][-3:]
    total = sum(s["correct"] + s["incorrect"] for s in recent)
    acc = sum(s["correct"] for s in recent) / max(1, total)
    if acc > 0.8:
        return "You're doing great! Try full word Braille practice."
    elif acc < 0.5:
        return "Focus on individual letter recognition and repeat past tests."
    return "You’re making good progress. Keep practicing mixed letters."

# ========== INTENT-BASED FUNCTION CALL ==========
def call_function_by_intent(user_input):
    prompt = f"""You are a Braille teaching assistant.
Understand user commands and classify their intent as one of these:
- translate
- quiz
- suggest
- qna
Only return the intent word.

Input: {user_input}
Intent:"""
    result = llm(prompt, num_return_sequences=1)[0]['generated_text'].strip().lower()
    return result

# ========== CHAT INTERFACE ==========
def ai_braille_chat():
    rprint("[bold green]Welcome to the AI Braille Assistant[/]")
    rprint("[bold yellow]Ask anything. Commands: translate, quiz, suggest, or Braille questions. Type 'exit' to quit.[/]\n")

    while True:
        user_input = input("You: ").strip()
        if user_input.lower() == "exit":
            rprint("[bold yellow]Goodbye![/]")
            break

        intent = call_function_by_intent(user_input)

        if "translate" in intent:
            text = input("Text to convert to Braille: ")
            lang = detect_language(text)
            braille = translate_to_braille(text, lang)
            rprint(f"[magenta]Detected: {lang}[/]\n[cyan]Braille Output:[/] {braille}")
            speak(f"Braille output is {braille}")

        elif "quiz" in intent:
            lang = input("Which language (english, tamil, etc.): ").lower()
            if lang not in braille_dicts:
                rprint("[red]Unsupported language![/]")
                continue
            letters = list(braille_dicts[lang].keys())
            sample = random.sample(letters, min(5, len(letters)))
            responses = {}
            for ch in sample:
                ans = input(f"Braille for '{ch}': ")
                responses[ch] = ans
            correct = sum(1 for ch in sample if responses[ch] == braille_dicts[lang][ch])
            update_performance(correct, len(sample) - correct, [ch for ch in sample if responses[ch] != braille_dicts[lang][ch]])
            rprint(f"[bold cyan]Quiz Score: {correct}/{len(sample)}[/]")

        elif "suggest" in intent:
            suggestion = get_suggestion()
            rprint(f"[bold blue]AI Suggestion:[/] {suggestion}")
            speak(suggestion)

        elif "qna" in intent or "question" in user_input.lower():
            answer = llm(user_input, num_return_sequences=1)[0]['generated_text']
            rprint(f"[bold green]Answer:[/] {answer}")
            speak(answer)

        else:
            reply = llm(user_input, num_return_sequences=1)[0]['generated_text']
            rprint(f"[bold white]Chat:[/] {reply}")

if __name__ == "__main__":
    ai_braille_chat()


Loading LLM model...

Device set to use cpu


Welcome to the AI Braille Assistant

Ask anything. Commands: translate, quiz, suggest, or Braille questions. Type 'exit' to quit.

You: hello bro


...

Detected: tamil
Braille Output: ⠥ங்களைச் சந்தித்ததில் மகிழ்ச்சி

You: what are the fuctionalities you have?
Text to convert to Braille: exit


Detected: english
Braille Output: ⠑xit

You: exit


Goodbye!

In [ ]:
from transformers import pipeline

# Load upgraded assistant LLM with chat conditioning
model = pipeline("text2text-generation", model="google/flan-t5-base", max_length=100)

# Chat conditioning prompt
def build_prompt(user_input):
    return f"""
You are a friendly AI assistant.
You can have conversations and also help users with tasks like:
- Translating text to Braille
- Starting Braille quizzes
- Suggesting what to learn next
- Answering Braille-related questions

If the user asks about Braille, do the task.
Otherwise, chat naturally.

User: {user_input}
AI:"""

# Example input
user_input = "What are your capabilities?"
prompt = build_prompt(user_input)

# Generate response
response = model(prompt, num_return_sequences=1)[0]['generated_text']
response


Device set to use cpu


'Answering Braille-related questions'

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

# Load Microsoft Phi-2 for local smart assistant
model_name = "microsoft/phi-2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Create text generation pipeline
phi2_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer)

# Prompt conditioning
def build_prompt(user_input):
    return f"""
You are an intelligent and helpful AI assistant.
You can:
- Have casual conversations
- Help users with Braille translation
- Start Braille quizzes
- Suggest what to study next
- Answer Braille questions

User: {user_input}
AI:"""

# Sample interaction
user_input = "What are you capable of?"
response = phi2_pipeline(build_prompt(user_input), max_new_tokens=100, do_sample=True)[0]["generated_text"]
response


tokenizer_config.json:   0%|          | 0.00/7.34k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/564M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Device set to use cpu
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [ ]:
# AI Braille Assistant — powered by Microsoft Phi-2, full offline LLM

import os
import json
import time
import random
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from gtts import gTTS
from rich import print as rprint

# ========== Load Phi-2 Model ==========
rprint("[yellow]Loading AI Model (Phi-2)...[/]")
phi_model = "microsoft/phi-2"
tokenizer = AutoTokenizer.from_pretrained(phi_model)
model = AutoModelForCausalLM.from_pretrained(phi_model, torch_dtype=torch.float32)
generator = pipeline("text-generation", model=model, tokenizer=tokenizer, device=-1)

# ========== Global Settings ==========
USER_DATA_FILE = "user_data.json"
SUPPORTED_LANGUAGES = ["english", "tamil", "hindi", "kannada", "bengali"]

# ========== Braille Mappings ==========
braille_dicts = {
    'english': {'a': '⠁', 'b': '⠃', 'c': '⠉', 'd': '⠙', 'e': '⠑', 'f': '⠋', 'g': '⠛', 'h': '⠓', 'i': '⠊', 'j': '⠚', 'k': '⠅', 'l': '⠇', 'm': '⠍', 'n': '⠝', 'o': '⠕', 'p': '⠏', 'q': '⠟', 'r': '⠗', 's': '⠎', 't': '⠞', 'u': '⠥', 'v': '⠧', 'w': '⠺', 'x': '⠭', 'y': '⠽', 'z': '⠵', ' ': ' '},
    'tamil': {'அ': '⠁', 'ஆ': '⠜', 'இ': '⠊', 'ஈ': '⠔', 'உ': '⠥', 'எ': '⠢', 'ஏ': '⠑', 'ஐ': '⠌', 'ஒ': '⠭', 'ஓ': '⠕', 'ஔ': '⠪', 'க': '⠅', 'ச': '⠉', 'ட': '⠾', 'த': '⠞', 'ப': '⠏', 'ம': '⠍', 'ய': '⠽', 'ர': '⠗', 'ல': '⠇', 'வ': '⠧', 'ழ': '⠷', 'ள': '⠸', 'ற': '⠻', 'ன': '⠰', ' ': ' '},
    'hindi': {'अ': '⠁', 'आ': '⠅', 'इ': '⠃', 'ई': '⠊', 'उ': '⠕', 'ऊ': '⠎', 'ऋ': '⠖', 'ए': '⠋', 'ऐ': '⠍', 'ओ': '⠕', 'औ': '⠕', 'क': '⠉', 'ग': '⠛', 'च': '⠉', 'ज': '⠐', 'ट': '⠘', 'त': '⠖', 'द': '⠘', 'न': '⠝', 'प': '⠛', 'फ': '⠋', 'ब': '⠘', 'म': '⠝', 'य': '⠽', 'र': '⠉', 'ल': '⠎', 'व': '⠗', 'श': '⠎', 'ह': '⠛', ' ': ' '},
    'kannada': {'ಅ': '⠁', 'ಆ': '⠜', 'ಇ': '⠊', 'ಈ': '⠔', 'ಉ': '⠕', 'ಊ': '⠾', 'ಋ': '⠗', 'ಎ': '⠑', 'ಐ': '⠜⠊', 'ಒ': '⠕', 'ಕ': '⠅', 'ಗ': '⠛', 'ಚ': '⠉', 'ಜ': '⠚', 'ಟ': '⠾', 'ಡ': '⠹', 'ತ': '⠹', 'ದ': '⠙', 'ನ': '⠝', 'ಪ': '⠏', 'ಬ': '⠃', 'ಮ': '⠍', 'ಯ': '⠽', 'ರ': '⠗', 'ಲ': '⠇', 'ವ': '⠺', 'ಶ': '⠩', 'ಸ': '⠎', 'ಹ': '⠓', ' ': ' '},
    'bengali': {'অ': '⠁', 'আ': '⠜', 'ই': '⠊', 'ঈ': '⠔', 'উ': '⠕', 'ঊ': '⠾', 'এ': '⠑', 'ঐ': '⠜⠊', 'ও': '⠕', 'ঔ': '⠕⠕', 'ক': '⠅', 'খ': '⠩', 'গ': '⠛', 'ঘ': '⠹', 'চ': '⠉', 'ছ': '⠡', 'জ': '⠚', 'ট': '⠾', 'ড': '⠹', 'ত': '⠹', 'দ': '⠙', 'ন': '⠝', 'প': '⠏', 'ফ': '⠋', 'ব': '⠃', 'ভ': '⠃⠃', 'ম': '⠍', 'য': '⠽', 'র': '⠗', 'ল': '⠇', 'শ': '⠩', 'স': '⠎', 'হ': '⠓', ' ': ' '}
}

# ========== Utilities ==========
def speak(text, lang='en'):
    tts = gTTS(text=text, lang=lang)
    tts.save("speak.mp3")
    os.system("start speak.mp3" if os.name == 'nt' else "afplay speak.mp3")

def detect_language(text):
    scores = {}
    for lang, d in braille_dicts.items():
        matches = sum(1 for char in text if char in d)
        scores[lang] = matches
    return max(scores, key=scores.get) if max(scores.values()) > 0 else 'english'

def translate_to_braille(text, lang):
    d = braille_dicts.get(lang, {})
    return ''.join(d.get(c, c) for c in text)

def load_user_data():
    if not os.path.exists(USER_DATA_FILE):
        return {"scores": [], "mistakes": {}}
    with open(USER_DATA_FILE, "r") as f:
        return json.load(f)

def save_user_data(data):
    with open(USER_DATA_FILE, "w") as f:
        json.dump(data, f, indent=2)

def update_performance(correct, incorrect, mistakes):
    data = load_user_data()
    data["scores"].append({"correct": correct, "incorrect": incorrect, "timestamp": time.time()})
    for ch in mistakes:
        data["mistakes"][ch] = data["mistakes"].get(ch, 0) + 1
    save_user_data(data)

def get_learning_suggestion():
    data = load_user_data()
    if not data["scores"]:
        return "Start with basic letters."
    last3 = data["scores"][-3:]
    correct = sum(x["correct"] for x in last3)
    total = sum(x["correct"] + x["incorrect"] for x in last3)
    acc = correct / max(1, total)
    if acc > 0.8:
        return "You're doing well! Try full word Braille modules."
    elif acc < 0.5:
        return "Focus on individual letters and repeat basics."
    else:
        return "Good progress! Mix letters and words for practice."

# ========== Assistant Interface ==========
def generate_reply(user_input):
    prompt = f"""You are a friendly AI assistant who can:
- Chat naturally
- Translate text to Braille
- Start quizzes
- Suggest learning
- Answer Braille questions

User: {user_input}
Assistant:"""
    return generator(prompt, max_new_tokens=100, do_sample=True)[0]['generated_text'].split("Assistant:")[-1].strip()

# ========== Main Chat Function ==========
def run_chatbot():
    rprint("[bold green]Welcome to your Smart Braille AI Assistant[/]")
    rprint("[bold yellow]Ask questions, chat, or say 'translate', 'quiz', 'suggest', 'exit'.[/]")

    while True:
        user_input = input("You: ").strip()
        if user_input.lower() == "exit":
            rprint("[bold red]Goodbye![/]")
            break

        # Detect intent and generate AI reply
        reply = generate_reply(user_input)
        rprint(f"[bold green]AI:[/] {reply}")
        lowered = user_input.lower()

        if "translate" in lowered:
            text = input("Enter text to convert to Braille: ")
            lang = detect_language(text)
            braille = translate_to_braille(text, lang)
            rprint(f"[magenta]{lang.title()} Braille Output:[/] {braille}")
            speak(braille)

        elif "quiz" in lowered:
            lang = input("Choose language (english/tamil/...): ").lower()
            if lang not in SUPPORTED_LANGUAGES:
                rprint("[red]Invalid language.[/]")
                continue
            keys = list(braille_dicts[lang].keys())
            sample = random.sample(keys, 5)
            correct = 0
            mistakes = []
            for ch in sample:
                ans = input(f"Braille for '{ch}': ").strip()
                if ans == braille_dicts[lang][ch]:
                    correct += 1
                else:
                    mistakes.append(ch)
            update_performance(correct, 5 - correct, mistakes)
            rprint(f"[cyan]Your score: {correct}/5[/]")

        elif "suggest" in lowered or "learn" in lowered:
            suggestion = get_learning_suggestion()
            rprint(f"[blue]AI Suggestion:[/] {suggestion}")
            speak(suggestion)

        elif any(w in lowered for w in ["what", "who", "how", "why", "when"]):
            rprint("[yellow]Asking AI...[/]")
            speak(reply)

# ========== Run ==========

if __name__ == "__main__":
    run_chatbot()


Loading AI Model (Phi-2)...

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cpu


Welcome to your Smart Braille AI Assistant

...

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


AI: Certainly! Braille is a writing system for the visually impaired and blind that uses raised dots to represent 
letters, numbers, punctuation marks, and other characters. It is read by touch with the fingertips

Asking AI...

You: exit


Goodbye!

In [ ]:
!pip install transformers torch gtts rich

In [ ]:
# FINAL: Braille AI Assistant with Hugging Face Phi-2

import os
import json
import time
import random
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from gtts import gTTS
from rich import print as rprint

# Load AI Model
rprint("[yellow]Loading Smart Assistant Model...[/]")
phi_model = "microsoft/phi-2"
tokenizer = AutoTokenizer.from_pretrained(phi_model)
model = AutoModelForCausalLM.from_pretrained(phi_model, torch_dtype=torch.float32)
generator = pipeline("text-generation", model=model, tokenizer=tokenizer, device=-1)
chat_history = []

# User progress file
USER_DATA_FILE = "user_data.json"

# Supported Languages
SUPPORTED_LANGUAGES = ["english", "tamil", "hindi", "kannada", "bengali"]

# Braille dictionaries (sample base)
braille_dicts = {
    'english': {'a': '⠁', 'b': '⠃', 'c': '⠉', 'd': '⠙', 'e': '⠑', 'f': '⠋', 'g': '⠛', 'h': '⠓', 'i': '⠊', 'j': '⠚', 'k': '⠅', 'l': '⠇', 'm': '⠍', 'n': '⠝', 'o': '⠕', 'p': '⠏', 'q': '⠟', 'r': '⠗', 's': '⠎', 't': '⠞', 'u': '⠥', 'v': '⠧', 'w': '⠺', 'x': '⠭', 'y': '⠽', 'z': '⠵', ' ': ' '},
    'tamil': {'அ': '⠁', 'ஆ': '⠜', 'இ': '⠊', 'ஈ': '⠔', 'உ': '⠥', 'எ': '⠢', 'ஏ': '⠑', 'ஐ': '⠌', 'ஒ': '⠭', 'ஓ': '⠕', 'ஔ': '⠪', 'க': '⠅', 'ச': '⠉', 'ட': '⠾', 'த': '⠞', 'ப': '⠏', 'ம': '⠍', 'ய': '⠽', 'ர': '⠗', 'ல': '⠇', 'வ': '⠧', 'ழ': '⠷', 'ள': '⠸', 'ற': '⠻', 'ன': '⠰', ' ': ' '},
    'hindi': {'अ': '⠁', 'आ': '⠅', 'इ': '⠃', 'ई': '⠊', 'उ': '⠕', 'ऊ': '⠎', 'ऋ': '⠖', 'ए': '⠋', 'ऐ': '⠍', 'ओ': '⠕', 'औ': '⠕', 'क': '⠉', 'ग': '⠛', 'च': '⠉', 'ज': '⠐', 'ट': '⠘', 'त': '⠖', 'द': '⠘', 'न': '⠝', 'प': '⠛', 'फ': '⠋', 'ब': '⠘', 'म': '⠝', 'य': '⠽', 'र': '⠉', 'ल': '⠎', 'व': '⠗', 'श': '⠎', 'ह': '⠛', ' ': ' '},
    'kannada': {'ಅ': '⠁', 'ಆ': '⠜', 'ಇ': '⠊', 'ಈ': '⠔', 'ಉ': '⠕', 'ಊ': '⠾', 'ಋ': '⠗', 'ಎ': '⠑', 'ಐ': '⠜⠊', 'ಒ': '⠕', 'ಕ': '⠅', 'ಗ': '⠛', 'ಚ': '⠉', 'ಜ': '⠚', 'ಟ': '⠾', 'ಡ': '⠹', 'ತ': '⠹', 'ದ': '⠙', 'ನ': '⠝', 'ಪ': '⠏', 'ಬ': '⠃', 'ಮ': '⠍', 'ಯ': '⠽', 'ರ': '⠗', 'ಲ': '⠇', 'ವ': '⠺', 'ಶ': '⠩', 'ಸ': '⠎', 'ಹ': '⠓', ' ': ' '},
    'bengali': {'অ': '⠁', 'আ': '⠜', 'ই': '⠊', 'ঈ': '⠔', 'উ': '⠕', 'ঊ': '⠾', 'এ': '⠑', 'ঐ': '⠜⠊', 'ও': '⠕', 'ঔ': '⠕⠕', 'ক': '⠅', 'খ': '⠩', 'গ': '⠛', 'ঘ': '⠹', 'চ': '⠉', 'ছ': '⠡', 'জ': '⠚', 'ট': '⠾', 'ড': '⠹', 'ত': '⠹', 'দ': '⠙', 'ন': '⠝', 'প': '⠏', 'ফ': '⠋', 'ব': '⠃', 'ভ': '⠃⠃', 'ম': '⠍', 'য': '⠽', 'র': '⠗', 'ল': '⠇', 'শ': '⠩', 'স': '⠎', 'হ': '⠓', ' ': ' '}
}

# Speak Output
def speak(text, lang='en'):
    tts = gTTS(text=text, lang=lang)
    tts.save("voice.mp3")
    os.system("start voice.mp3" if os.name == 'nt' else "afplay voice.mp3")

# Detect language
def detect_language(text):
    matches = {lang: sum(1 for ch in text if ch in braille_dicts[lang]) for lang in braille_dicts}
    return max(matches, key=matches.get) if max(matches.values()) > 0 else 'english'

# Translate to Braille
def translate_to_braille(text, lang):
    d = braille_dicts[lang]
    return ''.join(d.get(c, c) for c in text)

# Data Save/Load
def load_data():
    if os.path.exists(USER_DATA_FILE):
        with open(USER_DATA_FILE, "r") as f:
            return json.load(f)
    return {"scores": [], "mistakes": {}}

def save_data(data):
    with open(USER_DATA_FILE, "w") as f:
        json.dump(data, f, indent=2)

def update_score(correct, incorrect, mistakes):
    data = load_data()
    data["scores"].append({"correct": correct, "incorrect": incorrect, "time": time.time()})
    for ch in mistakes:
        data["mistakes"][ch] = data["mistakes"].get(ch, 0) + 1
    save_data(data)

def get_suggestion():
    data = load_data()
    if not data["scores"]:
        return "Start with the alphabet modules first."
    recent = data["scores"][-3:]
    total = sum(x["correct"] + x["incorrect"] for x in recent)
    correct = sum(x["correct"] for x in recent)
    acc = correct / max(1, total)
    if acc > 0.85:
        return "Great progress! Try full word Braille modules now."
    elif acc < 0.5:
        return "You should revisit the basics and practice letters."
    else:
        return "Good going! Mix words and letters for practice."

# Generate reply
def chat_response(user_input):
    context = "\n".join(chat_history[-6:])  # memory window size
    prompt = f"""
You are BRAILLEX — an intelligent, friendly, and helpful Braille transformer assistant.

You can:
- Have natural, engaging conversations like ChatGPT
- Understand and respond in any human language
- Translate any text into Braille for multiple languages
- Start Braille quizzes and track performance
- Suggest what the user should learn next based on progress
- Explain concepts, answer Braille-related questions, and be a mentor

Be warm, witty, supportive, and always guide the user.
{context}
User: {user_input}
BRAILLEX:"""

    response = generator(prompt, max_new_tokens=100, do_sample=True)[0]['generated_text']
    reply = response.split("BRAILLEX:")[-1].strip()

    # ✅ Append conversation to memory
    chat_history.append(f"User: {user_input}")
    chat_history.append(f"BRAILLEX: {reply}")

    return reply

# Main Chat
def run_assistant():
    rprint("[bold green]Welcome to Your Final-Year Braille AI Assistant[/]")
    rprint("[bold yellow]Ask me anything, or use commands like 'translate', 'quiz', 'suggest', or 'exit'.[/]\n")

    while True:
        user_input = input("You: ").strip()
        if user_input.lower() == "exit":
            rprint("[red]Goodbye![/]")
            break

        reply = chat_response(user_input)
        rprint(f"[bold green]AI:[/] {reply}")

        # Intent-based actions
        if "translate" in user_input.lower():
            text = input("Enter text to convert to Braille: ")
            lang = detect_language(text)
            output = translate_to_braille(text, lang)
            rprint(f"[bold cyan]{lang.upper()} Braille:[/] {output}")
            speak(f"Here is the Braille output: {output}")

        elif "quiz" in user_input.lower():
            lang = input("Choose quiz language (english/tamil/hindi/...): ").lower()
            if lang not in braille_dicts:
                rprint("[red]Language not supported[/]")
                continue
            sample = random.sample(list(braille_dicts[lang].keys()), 5)
            correct, mistakes = 0, []
            for ch in sample:
                ans = input(f"Braille for '{ch}': ")
                if ans == braille_dicts[lang][ch]:
                    correct += 1
                else:
                    mistakes.append(ch)
            update_score(correct, 5 - correct, mistakes)
            rprint(f"[bold magenta]Score: {correct}/5[/]")

        elif "suggest" in user_input.lower() or "learn" in user_input.lower():
            suggestion = get_suggestion()
            rprint(f"[bold blue]AI Suggestion:[/] {suggestion}")
            speak(suggestion)

        elif any(q in user_input.lower() for q in ["what", "who", "how", "why", "when"]):
            speak(reply)

# Run
if __name__ == "__main__":
    run_assistant()


Loading Smart Assistant Model...

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/7.34k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

You: hello dude!!


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


AI: Sure, I'd be happy to help you practice your Braille reading skills. Which topic would you like to focus on?


Consider four Braille learners - Alice, Bob, Charles, and Debra. Each of them are learning to read different 
subjects: Mathematics, Science, History, and English. There is also one AI software assistant each designed to 
focus on supporting in their respective

You: exit


Goodbye!